# Implementing a Viterbi Decoder and Evaluation for Sequence Labeling

In this assignment, you will build a Viterbi decoder for an LSTM named-entity recognition model. As we mentioned in class, recurrent and bidirectional recurrent neural networks, of which LSTMs are the most common examples, can be used to perform sequence labeling. Although these models encode information from the surrounding words in order to make predictions, there are no "hard" constraints on what tags can appear where.

There hard constraints are particularly important for tasks that label spans of more than one token. The most common example of a span-labeling task is named-entity recognition (NER). As described in Eisenstein, Jurafksy & Martin, and other texts, the goal of NER is to label spans of one or more words as _mentions_ of an _entity_, such as a person, location, organization, etc.

The most common approach to NER is to reduce it to a sequence-labeling task, where each token in the input is labeled either with an `O`, if it is "outside" any named-entity span, or with `B-TYPE`, if it is the first token in an entity of type `TYPE`, or with `I-TYPE`, if it is the second or later token in an entity of type `TYPE`. Distinguishing between the first and later tokens of an entity allow us to identify distinct entity spans even when they are adjacent.

Common values of `TYPE` include `PER` for person, `LOC` for location, `DATE` for date, and so on. In the dataset we load below, there are 17 distinct types.

The span-labeling scheme just described implies that the labels on tokens must obey certain constraints: the tag `I-PER` must follow either `B-PER` or another `I-PER`. It cannot follow `O`, `B-LOC`, or `I-LOC`, i.e., a tag for a different entity type. By themselves, LSTMs or bidirectional LSTMs cannot directly enforce these constraints. This is one reason why conditional random fields (CRFs), which _can_ enforce these constraints, are often layered on top of these recurrent models.

In this assignment, you will implement the simplest possible CRF: a CRF so simple that it does not require any training. Rather, it will assign weight 1 to any sequence of tags that obeys the constraints and weight 0 to any sequence of tags that violates them. The inputs to the CRF, which are analogous to the emission probabilities in an HMM, will come from an LSTM.

But first, in order to test your decoder, you will also implement some functions to evaluate the output of an NER system according to two metrics:
1. You will count the number of _violations_ of the NER label constraints, i.e., how many times `I-TYPE` follows `O` or a tag of a different type or occurs at the beginning of a sentence. This number will be greater than 0 in the raw LSTM output, but should be 0 for your CRF output.
1. You will compute the _span-level_ precision, recall, and F1 of NER output. Although the baseline LSTM was trained to achieve high _token-level_ accuracy, this metric can be misleadingly high, since so many tokens are correctly labeled `O`. In other words, what proportion of spans predicted by the model line up exactly with spans in the gold standard, and what proportion of spans in the gold standard were predicted by the model? Define _span_ as a sequence of tags that starts with a `B-TYPE` followed by zero or more `I-TYPE` tags. Sequences solely of `I-TYPE` tags don't count as spans.For more, see the original task definition: https://www.aclweb.org/anthology/W03-0419/.

We start with loading some code and data and the describe your tasks in more detail.

## Set Up Dependencies and Definitions

In [1]:
!pip install --upgrade spacy==2.1.0 allennlp==0.9.0
import spacy

     |████████████████████████████████| 27.7MB 102kB/s 
     |████████████████████████████████| 7.6MB 48.6MB/s 
     |████████████████████████████████| 2.1MB 46.2MB/s 
     |████████████████████████████████| 92kB 12.6MB/s 
     |████████████████████████████████| 3.2MB 52.8MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
     |████████████████████████████████| 133kB 52.0MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 133kB 55.3MB/s 
     |████████████████████████████████| 266kB 48.2MB/s 
     |████████████████████████████████| 317kB 54.2MB/s 
     |████████████████████████████████| 245kB 50.2MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 163kB 51.2MB/s 
     |████████████████████████████████| 5.6MB 53.7MB/s 
     |████████████████████████████████| 7.4MB 47.8MB/s 
     |████████████████████████████████| 81kB 10.0MB/s 
     |████████████████████████████████| 1.2MB 44.1MB/s 

In [2]:
from typing import Iterator, List, Dict
import torch
import torch.optim as optim
import numpy as np
from allennlp.data import Instance
from allennlp.data.fields import TextField, SequenceLabelField
from allennlp.data.dataset_readers import DatasetReader
from allennlp.common.file_utils import cached_path
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer
from allennlp.predictors import SentenceTaggerPredictor
from allennlp.data.dataset_readers import conll2003

torch.manual_seed(1)

In [3]:
class LstmTagger(Model):
  def __init__(self,
               word_embeddings: TextFieldEmbedder,
               encoder: Seq2SeqEncoder,
               vocab: Vocabulary) -> None:
    super().__init__(vocab)
    self.word_embeddings = word_embeddings
    self.encoder = encoder
    self.hidden2tag = torch.nn.Linear(in_features=encoder.get_output_dim(),
                                      out_features=vocab.get_vocab_size('labels'))
    self.accuracy = CategoricalAccuracy()

  def forward(self,
              tokens: Dict[str, torch.Tensor],
              metadata,
              tags: torch.Tensor = None) -> Dict[str, torch.Tensor]:
    mask = get_text_field_mask(tokens)
    embeddings = self.word_embeddings(tokens)
    encoder_out = self.encoder(embeddings, mask)
    tag_logits = self.hidden2tag(encoder_out)
    output = {"tag_logits": tag_logits}
    if tags is not None:
      self.accuracy(tag_logits, tags, mask)
      output["loss"] = sequence_cross_entropy_with_logits(tag_logits, tags, mask)

    return output

  def get_metrics(self, reset: bool = False) -> Dict[str, float]:
    return {"accuracy": self.accuracy.get_metric(reset)}

## Import Data

In [4]:
reader = conll2003.Conll2003DatasetReader()
train_dataset = reader.read(cached_path('http://www.ccs.neu.edu/home/dasmith/onto.train.ner.sample'))
validation_dataset = reader.read(cached_path('http://www.ccs.neu.edu/home/dasmith/onto.development.ner.sample'))

from itertools import chain
vocab = Vocabulary.from_instances(chain(train_dataset, validation_dataset))

159377B [00:00, 56449551.48B/s]
562it [00:00, 17865.96it/s]
8366B [00:00, 16343524.58B/s]
23it [00:00, 6795.03it/s]
585it [00:00, 69300.90it/s]


## Define and Train Model

In [5]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, bidirectional=False, batch_first=True))
model = LstmTagger(word_embeddings, lstm, vocab)
if torch.cuda.is_available():
    cuda_device = 0
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
# optimizer = optim.AdamW(model.parameters(), lr=1e-4, eps=1e-8)
optimizer = optim.SGD(model.parameters(), lr=0.1)
iterator = BucketIterator(batch_size=2, sorting_keys=[("tokens", "num_tokens")])
iterator.index_with(vocab)
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  patience=10,
                  num_epochs=100,
                  cuda_device=cuda_device)
trainer.train()

accuracy: 0.8442, loss: 0.9067 ||: 100%|██████████| 281/281 [00:01<00:00, 196.90it/s]
accuracy: 0.7878, loss: 1.2038 ||: 100%|██████████| 12/12 [00:00<00:00, 436.07it/s]
accuracy: 0.8442, loss: 0.7282 ||: 100%|██████████| 281/281 [00:01<00:00, 261.43it/s]
accuracy: 0.7878, loss: 1.1714 ||: 100%|██████████| 12/12 [00:00<00:00, 345.31it/s]
accuracy: 0.8442, loss: 0.7167 ||: 100%|██████████| 281/281 [00:01<00:00, 258.48it/s]
accuracy: 0.7878, loss: 1.1854 ||: 100%|██████████| 12/12 [00:00<00:00, 475.62it/s]
accuracy: 0.8442, loss: 0.7071 ||: 100%|██████████| 281/281 [00:01<00:00, 258.23it/s]
accuracy: 0.7878, loss: 1.1761 ||: 100%|██████████| 12/12 [00:00<00:00, 485.70it/s]
accuracy: 0.8442, loss: 0.6982 ||: 100%|██████████| 281/281 [00:01<00:00, 265.01it/s]
accuracy: 0.7878, loss: 1.1598 ||: 100%|██████████| 12/12 [00:00<00:00, 467.34it/s]
accuracy: 0.8442, loss: 0.6911 ||: 100%|██████████| 281/281 [00:01<00:00, 254.60it/s]
accuracy: 0.7878, loss: 1.1512 ||: 100%|██████████| 12/12 [00:00

{'best_epoch': 99,
 'best_validation_accuracy': 0.8775510204081632,
 'best_validation_loss': 0.371090716017837,
 'epoch': 99,
 'peak_cpu_memory_MB': 3230.692,
 'peak_gpu_0_memory_MB': 1060,
 'training_accuracy': 0.9263849168520284,
 'training_cpu_memory_MB': 3230.692,
 'training_duration': '0:02:00.117594',
 'training_epochs': 99,
 'training_gpu_0_memory_MB': 1060,
 'training_loss': 0.1901727049535694,
 'training_start_epoch': 0,
 'validation_accuracy': 0.8775510204081632,
 'validation_loss': 0.371090716017837}

## Evaluation

The simple code below loops over the validation set, applying the model to each exmaple and collecting out the input token, gold-standard output, and model output. You can see from these methods how to access ground truth and model outputs for evaluation.

In [6]:
def tag_sentence(s):
  tag_ids = np.argmax(model.forward_on_instance(s)['tag_logits'], axis=-1)
  fields = zip(s['tokens'], s['tags'], [model.vocab.get_token_from_index(i, 'labels') for i in tag_ids])
  return list(fields)

baseline_output = [tag_sentence(i) for i in validation_dataset]
## Show the first example
baseline_output[0]

[(With, 'O', 'O'),
 (a, 'O', 'O'),
 (wave, 'O', 'O'),
 (of, 'O', 'O'),
 (his, 'O', 'O'),
 (hand, 'O', 'O'),
 (,, 'O', 'O'),
 (Peng, 'B-PERSON', 'B-PERSON'),
 (Dehuai, 'I-PERSON', 'I-PERSON'),
 (said, 'O', 'O'),
 (that, 'O', 'O'),
 (despite, 'O', 'O'),
 (being, 'O', 'O'),
 (over, 'O', 'O'),
 (100, 'B-CARDINAL', 'B-CARDINAL'),
 (regiments, 'O', 'O'),
 (,, 'O', 'O'),
 (let, 'O', 'O'),
 ('s, 'O', 'O'),
 (call, 'O', 'O'),
 (this, 'O', 'O'),
 (campaign, 'O', 'O'),
 (the, 'B-EVENT', 'O'),
 (Hundred, 'I-EVENT', 'I-EVENT'),
 (Regiments, 'I-EVENT', 'I-EVENT'),
 (Offensive, 'I-EVENT', 'I-EVENT'),
 (., 'O', 'O')]

Now, you can implement two evaluation functions: `violations` and `span_stats`.

In [106]:
# TODO: count the number of NER label violations,
# such as O followed by I-TYPE or B-TYPE followed by
# I-OTHER_TYPE
# Take tagger output as input
def violations(tagged):
  count = 0
  for eachSentence in tagged:
    for i in range(len(eachSentence)):
        # check prediction token
        curTok = eachSentence[i][2]
        # if first item in the list is I, it's a violation
        if i == 0:
            if curTok.startswith("I"):
                count += 1
        else:
            # check last token if the current token is I
            if curTok.startswith("I"):
                lastTok = eachSentence[i - 1][2]
                if lastTok.startswith("O") or (lastTok.startswith("B") and curTok[2:] != lastTok[2:]) or (lastTok.startswith("I") and curTok[2:] != lastTok[2:]):
                    count += 1
  return count

# TODO: return the span-level precision, recall, and F1
# Only count valid spans that start with a B tag,
# followed by zero or more I tags of the same type.
# This is harsher than the token-level metric that the
# LSTM was trained to optimize, but it is the standard way
# of evaluating NER systems.
# Take tagger output as input
def span_stats(tagged):
  gold_spans = 0
  pred_spans = 0
  true_positive = 0

  for i in range(len(tagged)):   #check each sentence in tagged
    for j in range(len(tagged[i])):   #check each token in each sentence 
      if tagged[i][j][1].startswith('B'):  #if the true_tag of the token starts with B
        gold_spans += 1

      if tagged[i][j][2].startswith('B'):  #if the pred_tag  of the token starts with B    
        pred_spans += 1

      if tagged[i][j][1].startswith('B') and tagged[i][j][2].startswith('B'):
        if j < (len(tagged[i])-1):
          k = j + 1       #index of the next token.

        #if the next tag is an 'O' and it agrees with the gold standard -> true_positive increases 1:
        if tagged[i][k][2].startswith('O') and tagged[i][k][2] == tagged[i][k][1]:
          true_positive += 1

        #if the next tag is a 'B-Type' and it agrees with the gold standard -> true_positive increases 1::
        elif tagged[i][k][2].startswith('B') and tagged[i][k][2] == tagged[i][k][1]:
          true_positive += 1

        #if the next tag is an I-type
        elif tagged[i][k][2].startswith('I'):
          #continue checking the next tags if a tag is I-Type and has the same TYPE of the current B tag
          while tagged[i][k][2].startswith('I') and tagged[i][k][2][2:] == tagged[i][j][2][2:]:
            #check if the I-TYPE agrees with the gold standard:
            if tagged[i][k][2] == tagged[i][k][1]:          #I-org, I-org
              if k < (len(tagged[i])-1) : 
                k += 1
            #otherwise, it is invalid I-TYPE, thus the span is invalid.
            elif tagged[i][k][2] != tagged[i][k][1]:  # O,I-Org
              true_positive += 1
              break
            else: 
              break

          #the span ends at the end of the sentence
          if k == len(tagged[i]):
            true_positive += 1

          #end a span with an 'O' and agrees with the gold standard
          elif tagged[i][k][2] == tagged[i][k][1] and tagged[i][k][2].startswith('O'):
            true_positive += 1

          #start a new span and agrees with the gold standard
          elif tagged[i][k][2] == tagged[i][k][1] and tagged[i][k][2].startswith('B'):
            true_positive += 1
  
  print("true_positive", true_positive)
  print("gold: ", gold_spans)
  print("predicted: ",pred_spans)

  precision = true_positive/gold_spans
  recall = true_positive/pred_spans 
  f1 = (2*recall*precision)/(recall + precision)

  return {'precision': precision,
          'recall': recall,
          'f1': f1}

## You can check how many violations are made by the model output in predictor.
print(violations(baseline_output))

metrics = span_stats(baseline_output)

33
true_positive 12
gold:  43
predicted:  28


## Decoding

Now you can finally implement the simple Viterbi decoder. The `model` object, when applied to an input sentence, first calculates the scores for each possible output tag for each token. See the expression `model.forward_on_instance(s)['tag_logits']` in the code above.

Then, you will construct a transition matrix. You can use the code below to get a list of the tags the model knows about. For a set of K tags, construct a K-by-K matrix with a log(1)=0 when a transition between a given tag pair is valid and a log(0)=-infinity otherwise.

Finally, implement a Viterbi decoder that takes the model object and a dataset object and outputs tagged data, just like the `tag_sentence` function above. It should use the Viterbi algorithm with the (max, plus) semiring. You'll be working with sums of log probabilities instead of products of probabilties.

Run your `violations` function on the output of this decoder to make sure that there are no invalid tag transitions. Also, compare the span-level metrics on `baseline_output` and your new output using your `span_stats` function.

In [107]:
# This code shows how to map from output vector components to labels
ind_tag = vocab.get_index_to_token_vocabulary('labels') 

In [128]:
def create_transition_matrix():
  transition_matrix = np.zeros([len(ind_tag), len(ind_tag)])
  
  def is_a_violation(row_tag,col_tag):
    #row then col
    #valid cases:  B then I right type, B then O, O then B, B then O, I then I right type
    #invalid cases:B then I wrong type, O then I, I then I wrong type
    if col_tag.startswith("I"):
      if row_tag.startswith("O") or (row_tag.startswith("B") and col_tag[2:] != row_tag[2:]) or (row_tag.startswith("I") and col_tag[2:] != row_tag[2:]):
        return True
    return False

  for tag_index_1 in range(len(ind_tag)):
    for tag_index_2 in range(len(ind_tag)):
      if is_a_violation(ind_tag[tag_index_1], ind_tag[tag_index_2]): 
        transition_matrix[tag_index_1][tag_index_2] = np.log(0)
      else:
        transition_matrix[tag_index_1][tag_index_2] = np.log(1)
  return transition_matrix.transpose()



def viterbi_decoding(s):
    '''
      reference: https://ocw.mit.edu/courses/electrical-engineering-and-computer-science/6-096-algorithms-for-computational-biology-spring-2005/lecture-notes/lecture7.pdf
    '''
    #create transition matrix 
    transition_matrix = create_transition_matrix()

    #create transition matrix
    emission_matrix = model.forward_on_instance(s)['tag_logits']
    emission_matrix = emission_matrix.transpose()
    tag_num = emission_matrix.shape[0]
    word_num = emission_matrix.shape[1]

    #initialize dp_table to keep track the max values 
    viterbi = np.zeros([tag_num, word_num])    #34, num_of_word
    #initialize a sequence list to keep track the indices of the max values
    tag_sequence = []
    tag_indices = [0]*word_num   #list of indices corresponding to each word in a sentence
    
    # assign initial weights to the first word. 
    initial_weight = np.add(emission_matrix[:, 0], transition_matrix)
    viterbi[:, 0] = initial_weight[:,0]
    
    for t in range(1, word_num):
        temp_viterbi = np.add(viterbi[:, t-1], transition_matrix)  #add each column(word) to the transition matrix. Output is 34x34 matrix
        max_value = temp_viterbi.max(1)    #array of max values in the temp_viterbi matrix
        max_indices = temp_viterbi.argmax(1)   #array of indices of the above max values in the temp_viterbi matrix
        
        viterbi[:, t] = np.add(max_value, emission_matrix[:, t])     #add the max value array with the emission matrix then update the current column(word).
        tag_sequence.append(max_indices)   #add each array of indices to the seqquence list 

    #get the index of the largest log of a tag from the last words     
    index = viterbi[:, word_num - 1].argmax()
    
    #tracing backward the index of the most likely tag for each word 
    for k in range(word_num - 1, -1 , -1):
        prev_tag = tag_sequence[k - 1][index]
        tag_indices[k] = index
        index = prev_tag

    best_score  = np.max(viterbi[:, word_num - 1])   
    return best_score, tag_indices

In [129]:
my_baseline2 = []
for s in validation_dataset:
   
  score, seq = viterbi_decoding(s)
  fields = zip(s['tokens'], s['tags'], [model.vocab.get_token_from_index(i, 'labels') for i in seq])
  my_baseline2.append(list(fields))

res2 = violations(my_baseline2)
stats2 = span_stats(my_baseline2)
print('volations: {}'.format(res2))
print(stats2)

true_positive 33
gold:  43
predicted:  45
volations: 0
{'precision': 0.7674418604651163, 'recall': 0.7333333333333333, 'f1': 0.7499999999999999}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()
